**Assignment 6**

**Image and Video Analytics**

**Vishali Sharma**

**21MIA1066**

**Task 4: Number of People Entering and Exiting the Shop**
    
**Objective:**
    
**Count the number of people entering and exiting a shop based on video footage, using basic motion detection techniques.**

**Task Description:**

**Load Video:**

**Load the provided surveillance video of the shop entrance.**

**Motion Detection:**

Use frame differencing or optical flow to detect motion as people enter and exit the shop.*
Define a region of interest (ROI) near the entrance to focus on counting people.
Counting People:

Track the direction of motion (inward or outward) based on detected motion in the ROI.
Increment a counter for each person entering and exiting.
Result:

Display the total number of people who entered and exited the shop during the recorded period.

In [2]:
import cv2
import numpy as np


In [3]:
#task-4:
# Load the video
video_path = r"shoppinginndout.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Define region of interest (ROI) near the entrance
roi_x, roi_y, roi_width, roi_height = 445, 200, 200, 150  # Adjust based on entrance position

# Background subtraction for motion detection
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50)

# Initialize counters for people entering and exiting
enter_count = 0
exit_count = 0

# Variables to hold movement direction
last_direction = None
direction_threshold = 30  # Minimum movement threshold for counting

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Define the ROI for detecting motion at the entrance
    roi = frame[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)

    # Detect motion using background subtraction
    fg_mask = fgbg.apply(blurred_roi)
    _, fg_mask = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

    # Find contours to identify moving objects
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Ignore small contours to avoid noise
        if cv2.contourArea(contour) < 500:
            continue

        # Draw bounding box around detected motion in the ROI
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(roi, (x, y), (x+w, y+h), (100, 255, 100), 2)

        # Calculate movement direction (up for entering, down for exiting)
        if last_direction is None:
            last_direction = y
        else:
            direction = y - last_direction
            if abs(direction) > direction_threshold:
                if direction < 0:
                    enter_count += 1
                    #print(f"Person entered, Total Entered: {enter_count}")
                elif direction > 0:
                    exit_count += 1
                    #print(f"Person exited, Total Exited: {exit_count}")
                last_direction = y

    # Display the frame with ROI and motion highlighted
    cv2.rectangle(frame, (roi_x, roi_y), (roi_x+roi_width, roi_y+roi_height), (255, 0, 0), 2)
    cv2.putText(frame, f"Entered: {enter_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Exited: {exit_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.imshow("Shop Entrance", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

print(f"Final count - Entered: {enter_count}, Exited: {exit_count}")

Final count - Entered: 1, Exited: 2
